In [4]:
import string
import csv
import collections
import nltk
from nltk.corpus import stopwords
import operator
from collections import OrderedDict
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io
import os
import math

english_stops = []

def clean_tokens(tokens):
    """ Lowercases, takes out punct and stopwords and short strings """
    return [token.lower() for token in tokens if (token not in string.punctuation) and
               	(token.lower() not in english_stops) and len(token) > 2]

def get_stopwords():
    enc = 'utf-8'
    with open('stopword_file.csv', 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    english_stops = [i[0] for i in keywords]
    #print ( english_stops)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
def get_cleanTokens(directory):
    cleanTokens = []
    for filename in os.listdir(directory):
        text = get_text("KeywordDocs/" + filename)
        tokens = nltk.word_tokenize(text)
        cleanTokens.extend(clean_tokens(tokens))
    return cleanTokens
     
def create_freqList(cleanTokens):
    wordcount = {} 
    for tok in cleanTokens:
        if tok not in wordcount:
            wordcount[tok] = 1
        else:
            wordcount[tok] += 1
        
    sorted_wordCount = OrderedDict(sorted(wordcount.items(), key=operator.itemgetter(1), reverse=True))
    write_csv(sorted_wordCount)

def write_csv(sorted_wordCount):
    enc = 'utf-8'
    if not os.path.isfile('Data/frequency.csv'):
        with open('Data/frequency.csv', 'w', encoding = enc) as f:
            columnTitleRow = "Word, Count\n"
            f.write(columnTitleRow)
            for key,value in sorted_wordCount.items():
                f.write("%s, %d\n"%(key,value))
    else:
        with open('Data/frequency.csv', 'a+', encoding = enc) as f:
            for key,value in sorted_wordCount.items():
                f.write("%s,\n"%(key,value))


def get_text(filename):
    fp = open(filename, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data

if __name__ == '__main__':
    get_stopwords()
    clean_tokens = get_cleanTokens("KeywordDocs")
    create_freqList(clean_tokens)